#### Primeiro request (de varios)

In [1]:
import requests
import csv
from threading import Thread
from multiprocessing.pool import ThreadPool
import pandas as pd

url = 'http://ws.audioscrobbler.com/2.0'
api_key = 'e280570373602815f884f71abba1c95f'
shared_secret = 'de39e3489affd5d12e20e7ce37a3c9cb'

#### Loop de Amigos
output: lista usernames com todos usernames iterados

In [ ]:
activeUserList = []
with open('activeUserList_1.csv', 'r', newline= '') as f:
    reader = csv.reader(f)
    for row in reader:
        for user in row:
            activeUserList.append(user)
    print(activeUserList)
    print(len(activeUserList))


In [2]:
def isActiveUser(username):
    params = {
        'api_key': api_key,
        'method': 'user.getRecentTracks',
        'user': username,
        'format': 'json',
        'limit': 1
    }

    response = requests.get(url, params=params)
    dataActiveUser = response.json()

    try:
        if int(dataActiveUser['recenttracks']['track'][0]['date']['uts']) > 1640995200:
            print(f'user approved: {username}')
            return True
        else:
            print(f'user rejected: {username}')
            return False

    except:
        return False


In [ ]:
activeUserList = ['user']

TOTAL_USER_NUMBER = 5000
userCount = 1

for username in activeUserList:
    newUsernames = []

    params = {
        'api_key': api_key,
        'method': 'user.getfriends',
        'user': username,
        'format': 'json'
    }

    response = requests.get(url, params=params)
    dataFriends = response.json()
    try:
        for friend in dataFriends['friends']['user']:
            if ((friend['name'] not in activeUserList) and isActiveUser(friend['name'])):
                newUsernames.append(friend['name'])
                userCount += 1

                if (userCount >= TOTAL_USER_NUMBER):
                    break
            else:
                pass
    except Exception as e:
        print("Failed to import user list. Exception: ", e)
        pass
    
    activeUserList.extend(newUsernames)
    if (userCount >= TOTAL_USER_NUMBER):
        break

with open('activeUserList.csv', 'w', newline= '\n') as f:
    writer = csv.writer(f)
    writer.writerow(activeUserList)

print(activeUserList)
print(len(activeUserList))

### Criando o dataframe

Para cada usuario aprovado de active_users, usa o método user.getRecentTracks para pegar os ultimos scrobbles de cada usuario.

OUTPUT: Lista de dicionarios activeUserScrobbles com os ultimos scrobbles de cada usuario em active_users

In [4]:
def getUserScrobbles(user):
       params = {
              'api_key': api_key,
              'method': 'user.getRecentTracks',
              'user': user,
              'format': 'json',
              'limit': 100
       }  
       response = requests.get(url, params=params)
       data = response.json()

       scrobbles = []

       try:
                     
              for scrobble in data['recenttracks']['track']:

                     aux = {'User Id':user,
                            'Artist Name':scrobble['artist']['#text'],
                            'Track ID':scrobble['mbid'],
                            'Album ID': scrobble['album']['mbid'],
                            'Album Name': scrobble['album']['#text'],
                            'Track Name': scrobble['name'],
                            'Artist ID': scrobble['artist']['mbid']}
                     
                     scrobbles.append(aux)
                     print(f'scrobble saved: ',aux)

       except:
              print(f'user {user} has missing recent track data')

       return(scrobbles)





In [ ]:
activeUserScrobbles = []
count = 0

with ThreadPool(10) as activeUserPool:
    for result in activeUserPool.map(getUserScrobbles, activeUserList):
        for item in result:
            activeUserScrobbles.append(item)

### Utiliza diversos métodos para pegar diversas informações de cada scrobble da lista de scrobble

In [6]:
def getTracksTags(scrobble):
    trackTagsParams = {
        'api_key': api_key,
        'method': 'track.getInfo',
        'artist':scrobble['Artist Name'],
        'track': scrobble['Track Name'],
        'format': 'json'
    }
    
    response = requests.get(url, params=trackTagsParams)
    data = response.json()

    trackTags = []
    try:
        trackDuration = data['track']['duration']
        trackListeners = data['track']['listeners']
        trackPlaycount = data['track']['playcount']

        if (len(data['track']['toptags']) > 0):
            for tag in data['track']['toptags']['tag']:
                if(len(tag) == 2):
                    trackTags.append(tag['name'])
                else:
                   trackTags.append('') 
        else:
            trackTags.append('')

        scrobble['Track Duration'] = trackDuration
        scrobble['Track Listeners'] = trackListeners
        scrobble['Track Playcount'] = trackPlaycount
        scrobble['Track Tags'] = trackTags

    except:
        scrobble['Track Duration'] = ''
        scrobble['Track Listeners'] = ''
        scrobble['Track Playcount'] = ''
        scrobble['Track Tags'] = ''

def getAlbumTags(scrobble):
    albumTagsParams = {
        'api_key': api_key,
        'method': 'album.getInfo',
        'artist': scrobble['Artist Name'],
        'album': scrobble['Album Name'],
        'format': 'json'
    }

    response = requests.get(url, params=albumTagsParams)
    data = response.json()

    albumTags = []
    try:
        albumListeners = data['album']['listeners']
        albumPlaycount = data['album']['playcount']

        if (len(data['album']['tags']) > 0):
            for tag in data['album']['tags']['tag']:
                if(len(tag) == 2):
                    albumTags.append(tag['name'])
                else:
                    albumTags.append('')
        else:
            albumTags.append('')

        scrobble['Album Listeners'] = albumListeners
        scrobble['Album Playcount'] = albumPlaycount
        scrobble['Album Tags'] = albumTags
        
    except KeyError:
        scrobble['Album Listeners'] = ''
        scrobble['Album Playcount'] = ''
        scrobble['Album Tags'] = ''

        
def getArtistTags(scrobble):
    artistTagsParams = {
        'api_key': api_key,
        'method': 'artist.getInfo',
        'artist':scrobble['Artist Name'],
        'format': 'json'
    }

    response = requests.get(url, params=artistTagsParams)
    data = response.json()

    artistTags = []
    try:
        artistListeners = data['artist']['stats']['listeners']
        artistPlaycount = data['artist']['stats']['playcount']

        if (len(data['artist']['tags']) > 0):
            for tag in data['artist']['tags']['tag']:
                if(len(tag) == 2):
                    artistTags.append(tag['name'])
                else:
                    artistTags.append('')
        else:
            artistTags.append('')

        scrobble['Artist Listeners'] = artistListeners
        scrobble['Artist Playcount'] = artistPlaycount
        scrobble['Artist Tags'] = artistTags

    except:
        scrobble['Artist Listeners'] = ''
        scrobble['Artist Playcount'] = ''
        scrobble['Artist Tags'] = ''

def getSimilarArtists(scrobble):
    similarArtistsParams = {
        'api_key': api_key,
        'method': 'artist.getsimilar',
        'artist': scrobble['Artist Name'],
        'format': 'json'
    }

    response = requests.get(url, params=similarArtistsParams)
    data = response.json()

    similarArtists = []
    try:
        for artist in data['similarartists']['artist']:
            similarArtists.append(artist['name'])

        scrobble['Similar Artists'] = similarArtists

    except:
        scrobble['Similar Artists'] = []

def getArtistTopTracks(scrobble):
    artistTopTracksParams = {
        'api_key': api_key,
        'method': 'artist.getTopTracks',
        'artist': scrobble['Artist Name'],
        'limit': 10,
        'format': 'json'
    }
    
    response = requests.get(url, params=artistTopTracksParams)
    data = response.json()
    
    top_tracks = []
    try:
        for track in data['toptracks']['track']:
            top_tracks.append(track['name'])

        scrobble['Artist Top Tracks'] = top_tracks

    except:
        scrobble['Artist Top Tracks'] = []
    

def getArtistTopAlbums(scrobble):
    artistTopAlbumsParams = {
        'api_key': api_key,
        'method': 'artist.getTopAlbums',
        'artist': scrobble['Artist Name'],
        'limit': 10,
        'format': 'json'
    }
    
    response = requests.get(url, params=artistTopAlbumsParams)
    data = response.json()
    
    top_albums = []
    try:
        for album in data['topalbums']['album']:
            top_albums.append(album['name'])

        scrobble['Artist Top Albums'] = top_albums

    except:
        scrobble['Artist Top Albums'] = []


In [ ]:
def getScrobbleData(scrobble):
    trackTagsThread = Thread(target=getTracksTags, args=[scrobble])
    trackTagsThread.start()

    albumTagsThread = Thread(target=getAlbumTags, args=[scrobble])
    albumTagsThread.start()

    artistTagsThread = Thread(target=getArtistTags, args=[scrobble])
    artistTagsThread.start()

    similarArtistThread = Thread(target=getSimilarArtists, args=[scrobble])
    similarArtistThread.start()

    artistTopTracksThread = Thread(target=getArtistTopTracks, args=[scrobble])
    artistTopTracksThread.start()

    artistTopAlbumsThread = Thread(target=getArtistTopAlbums, args=[scrobble])
    artistTopAlbumsThread.start()

    trackTagsThread.join()
    albumTagsThread.join()
    artistTagsThread.join()
    similarArtistThread.join()
    artistTopTracksThread.join()
    artistTopAlbumsThread.join()


for scrobble in activeUserScrobbles:
    getScrobbleData(scrobble)

### Consolida dataframe com as informações de activeUsersScrobbles e exporta para um arquivo csv

In [ ]:
df =pd.DataFrame(activeUserScrobbles)
df.to_csv('DatasetCompleto.csv', index=False)